In [1]:
import pandas as pd
from pandas import *
import numpy as np

from datetime import datetime

# Get the training data
df = pd.read_csv("data/train.csv.gz")

# TODO Read test data

# Just get a subset for fast results
#df = df[1:100000]

# Show first 5 columns
df.head(5)
df.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [2]:
# convert day of week into 3 booleans, so that hidden layers of neural network do the rest
def day_of_week_and_month_and_year(date):
    datetime_object = datetime.strptime(date, "%Y-%m-%d")
    week_day = datetime_object.weekday()
    
    # Same for month
    month = datetime_object.month
    year = datetime_object.year
    
    return (week_day, month, year)

df['dow'], df['month'], df['year'] = zip(*df["date"].map(day_of_week_and_month_and_year))
# Column not needed anymore
#df = df.drop(['date'], axis=1)

# Rearrange order of dataframe
df = df[['date', 'store', 'item', 'dow', 'month', 'year', 'sales']]

#df['sales'] = df['sales'].as_type('int')
df.head(5)
df.dtypes

date     object
store     int64
item      int64
dow       int64
month     int64
year      int64
sales     int64
dtype: object

In [3]:
# Prepare result values + 

In [4]:
# Generate our training/validation datasets
from sklearn import model_selection


array = df.values

X = array[:,1:6]
# Get sales
Y = array[:,6]

validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)
print(X)
Y_train=Y_train.astype('int')

[[1 1 1 1 2013]
 [1 1 2 1 2013]
 [1 1 3 1 2013]
 ..., 
 [10 50 4 12 2017]
 [10 50 5 12 2017]
 [10 50 6 12 2017]]


In [5]:
# Import algorithm
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC




# Test options and evaluation metric
seed = 7
scoring = 'accuracy'

# Spot Check Algorithms
models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []

#Y_train=Y_train.astype('int')

for name, model in models:
    print("Executing for model {0}".format(name))
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [7]:
# Executing for cart only

name = 'CART'
model = DecisionTreeClassifier()
print("Executing for model {0}".format(name))
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
results.append(cv_results)
names.append(name)
msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(msg)

Executing for model CART
CART: 0.042885 (0.000482)
